In [1]:
import pandas as pd
import numpy as np
import requests

from pandas.io.json import json_normalize

!pip install lxml
import lxml

     |████████████████████████████████| 5.8MB 21.7MB/s eta 0:00:01�       | 4.5MB 21.7MB/s eta 0:00:01


In [2]:
c = pd.read_csv('cities15000m.csv',sep=';')
c.head()

,geonameid,name,latitude,longitude,country code,population,timezone
0,2650225,Edinburgh,55.95206,-3.19640,GB,464990,Europe/London
1,2648579,Glasgow,55.86515,-4.25763,GB,591620,Europe/London
2,1841603,Gyeongju,35.84278,129.21167,KR,155237,Asia/Seoul
3,4726206,San Antonio,29.42412,-98.49363,US,1469845,\tAmerica/Chicago
4,3040051,les Escaldes,42.50729,1.53414,AD,15853,Europe/Andorra


In [3]:
c.loc[c['country code']=='SE'].loc[c.name.str.contains('G')]

,geonameid,name,latitude,longitude,country code,population,timezone
18318,2709628,Gustavsberg,59.32684,18.38975,SE,20774,Europe/Stockholm
18319,2711537,Goeteborg,57.70716,11.96679,SE,572799,Europe/Stockholm
18320,2712414,Gaevle,60.67452,17.14174,SE,74884,Europe/Stockholm
18321,2712993,Gamla Uppsala,59.89817,17.63386,SE,16231,Europe/Stockholm


In [4]:
c.loc[c['country code']=='CN'].loc[c.name.str.contains('L')].loc[c.name.str.contains('ou')]

,geonameid,name,latitude,longitude,country code,population,timezone
3209,1792516,Loushanguan,28.13680,106.82200,CN,80344,Asia/Shanghai
3358,1802238,Loudi,27.73444,111.99444,CN,150684,Asia/Shanghai
3382,1803841,Lianzhou,24.78186,112.37120,CN,92827,Asia/Shanghai
3383,1803842,Lianzhou,21.66621,109.20116,CN,71423,Asia/Shanghai
3393,1804386,Laohekou,32.38583,111.66778,CN,253112,Asia/Shanghai
3394,1804430,Lanzhou,36.05701,103.83987,CN,2628426,Asia/Shanghai
3715,2036066,Linkou,45.28606,130.26151,CN,77754,Asia/Shanghai
3724,2036241,Langtou,40.02611,124.32861,CN,59046,Asia/Shanghai


In [5]:
#Washington DC adjustment
c.loc[c.name.str.contains('D.C.'), 'name'] = 'Washington DC'

# Chiang Mai not joining for some reason
c.loc[c.name.str.contains('Chiang Mai'), 'name'] = 'Chiang Mai' 

# Found:
# Edinburgh, Ganzhou, Glasgow, Gothenburg, Gyeongju

# Missing cities: 
# Liuzhou Ma'anshan Pu'er,
# Queenstown, Ramstein, San Antonio
# Sharm El Sheikh, Tagaytay, Tulum

In [237]:
c.sort_values('population',ascending=False).tail(100)

,geonameid,name,latitude,longitude,country code,population,timezone
11087,1277202,Bantva,21.48815,70.07576,IN,15051,Asia/Kolkata
17733,575457,Boguchar,49.93577,40.54500,RU,15048,Europe/Moscow
22095,5315062,Somerton,32.59644,-114.70968,US,15048,America/Phoenix
16244,1180454,Darya Khan,31.79123,71.10394,PK,15048,Asia/Karachi
20176,4175296,Three Lakes,25.64205,-80.39839,US,15047,America/New_York
...,...,...,...,...,...,...,...
5374,7302786,Haselbachtal,51.23570,14.02576,DE,0,Europe/Berlin
16796,7303944,Melekeok,7.50043,134.62355,PW,0,Pacific/Palau
916,2800461,Chasse Royale,50.42842,3.95001,BE,0,Europe/Brussels
8254,4043547,Guam Government House,13.47191,144.74978,GU,0,Pacific/Guam


In [6]:
cs = c.sort_values('population', ascending=False)

In [7]:
cs.columns

Index(['geonameid', 'name', 'latitude', 'longitude', 'country code',
       'population', 'timezone'],
      dtype='object')

In [8]:
cs.columns = ['geonameid', 'city', 'latitude', 'longitude', 'country code',
       'population', 'timezone']
cs.head()

,geonameid,city,latitude,longitude,country code,population,timezone
3266,1796236,Shanghai,31.22222,121.45806,CN,22315474,Asia/Shanghai
19289,745044,Istanbul,41.01384,28.94966,TR,14804116,Europe/Istanbul
194,3435910,Buenos Aires,-34.61315,-58.37723,AR,13076300,America/Argentina/Buenos_Aires
10909,1275339,Mumbai,19.07283,72.88261,IN,12691836,Asia/Kolkata
14207,3530597,Mexico City,19.42847,-99.12766,MX,12294193,America/Mexico_City


In [9]:
cg = cs.groupby(cs['city']).first()

In [10]:
cg = cg.reset_index()

In [11]:
cg.loc[cg.city == 'Albany']

,city,geonameid,latitude,longitude,country code,population,timezone
496,Albany,5106834,42.65258,-73.75623,US,98469,America/New_York


In [12]:
cg.index = cg.city

In [13]:
cg.head()

,city,geonameid,latitude,longitude,country code,population,timezone
city,,,,,,,
'Afak,'Afak,99738,32.06430,45.24743,IQ,21888,Asia/Baghdad
'Ain Abid,'Ain Abid,2508309,36.23194,6.94333,DZ,29486,Africa/Algiers
'Ain Benian,'Ain Benian,2508275,36.80277,2.92185,DZ,31102,Africa/Algiers
'Ain Deheb,'Ain Deheb,2508225,34.84218,1.54697,DZ,36146,Africa/Algiers
'Ain Merane,'Ain Merane,2508010,36.16277,0.97037,DZ,34413,Africa/Algiers


In [14]:
cn = pd.read_html('https://www.geonames.org/countries/')[1]

In [15]:
cn.head()

,ISO-3166alpha2,ISO-3166alpha3,ISO-3166numeric,fips,Country,Capital,Area in km²,Population,Continent
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,4975593,AS
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,29121286,AS
3,AG,ATG,28,AC,Antigua and Barbuda,St. John's,443.0,86754,NaN
4,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,NaN


In [16]:
cn.columns
cn.columns = ['code', 'ISO-3166alpha3', 'ISO-3166numeric', 'fips',
       'Country', 'Capital', 'Area in km²', 'Population', 'Continent']

In [17]:
cn.loc[cn['Country'].str.contains('Mala')] 

,code,ISO-3166alpha3,ISO-3166numeric,fips,Country,Capital,Area in km²,Population,Continent
157,MW,MWI,454,MI,Malawi,Lilongwe,118480.0,17563749,AF
159,MY,MYS,458,MY,Malaysia,Kuala Lumpur,329750.0,28274729,AS


In [18]:
cn.index = cn.code

In [19]:
cg.head()

,city,geonameid,latitude,longitude,country code,population,timezone
city,,,,,,,
'Afak,'Afak,99738,32.06430,45.24743,IQ,21888,Asia/Baghdad
'Ain Abid,'Ain Abid,2508309,36.23194,6.94333,DZ,29486,Africa/Algiers
'Ain Benian,'Ain Benian,2508275,36.80277,2.92185,DZ,31102,Africa/Algiers
'Ain Deheb,'Ain Deheb,2508225,34.84218,1.54697,DZ,36146,Africa/Algiers
'Ain Merane,'Ain Merane,2508010,36.16277,0.97037,DZ,34413,Africa/Algiers


In [20]:
cn.head()

,code,ISO-3166alpha3,ISO-3166numeric,fips,Country,Capital,Area in km²,Population,Continent
code,,,,,,,,,
AD,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU
AE,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,4975593,AS
AF,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,29121286,AS
AG,AG,ATG,28,AC,Antigua and Barbuda,St. John's,443.0,86754,NaN
AI,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,NaN


In [21]:
# Czech Republic adjustment
cn.loc[cn['code']=='CZ','Country'] = 'Czech Republic'

In [22]:
cities = c.join(cn[['Country','Capital','Population','Continent']], on='country code')

In [23]:
c2 = cities.sort_values('population', ascending=False).groupby(['name','Country']).first().reset_index()

In [24]:
#c2.loc[c2['geonameid']==3078610]
#c2.loc[c2['geonameid']==1153671]
c2.loc[c2['geonameid']==2643743]

,name,Country,geonameid,latitude,longitude,country code,population,timezone,Capital,Population,Continent
11494,London,United Kingdom,2643743,51.50853,-0.12574,GB,7556900,Europe/London,London,62348447,EU


In [25]:
c2.shape

(22816, 11)

In [26]:
c2.columns = ['city', 'country', 'geonameid', 'latitude', 'longitude', 'country code',
       'population', 'timezone', 'Capital', 'Population', 'Continent']
c2 = c2.set_index(['city','country'])

In [27]:
c2.head()

,,geonameid,latitude,longitude,country code,population,timezone,Capital,Population,Continent
city,country,,,,,,,,,
'Afak,Iraq,99738,32.06430,45.24743,IQ,21888,Asia/Baghdad,Baghdad,29671605,AS
'Ain Abid,Algeria,2508309,36.23194,6.94333,DZ,29486,Africa/Algiers,Algiers,34586184,AF
'Ain Benian,Algeria,2508275,36.80277,2.92185,DZ,31102,Africa/Algiers,Algiers,34586184,AF
'Ain Deheb,Algeria,2508225,34.84218,1.54697,DZ,36146,Africa/Algiers,Algiers,34586184,AF
'Ain Merane,Algeria,2508010,36.16277,0.97037,DZ,34413,Africa/Algiers,Algiers,34586184,AF


In [28]:
from datetime import datetime

#strptime sample
dstr = '{} {}'.format('January', 2018)
dt = datetime.strptime(dstr, '%B %Y')
print(dt)

2018-01-01 00:00:00


In [29]:
# Read mugs data
m = pd.read_json('mugs.json', orient='index')
m = m.reset_index().drop(columns=['badge','icon','image','imageBy'])
m.columns = ['id', 'city', 'country', 'month', 'name', 'series', 'state', 'year'] 
m.head()

,id,city,country,month,name,series,state,year
0,aachen_icon,Aachen,Germany,January,Aachen,icon,NaN,2009
1,aarhus_icon,Aarhus,Denmark,January,Aarhus,icon,NaN,2014
2,aberdeen_yah,Aberdeen,United Kingdom,August,Aberdeen,yah,Scotland,2017
3,abu_dhabi_icon,Abu Dhabi,United Arab Emirates,January,Abu Dhabi,icon,NaN,2011
4,acapulco_bts,Acapulco,Mexico,August,Acapulco,bts,Guerrero,2018


In [30]:
# Add Date column
m['date'] = (m['month'] + ' ' + m['year'].astype(str)).apply(lambda x: datetime.strptime(x, '%B %Y'))

In [31]:
m.head()

,id,city,country,month,name,series,state,year,date
0,aachen_icon,Aachen,Germany,January,Aachen,icon,NaN,2009,2009-01-01
1,aarhus_icon,Aarhus,Denmark,January,Aarhus,icon,NaN,2014,2014-01-01
2,aberdeen_yah,Aberdeen,United Kingdom,August,Aberdeen,yah,Scotland,2017,2017-08-01
3,abu_dhabi_icon,Abu Dhabi,United Arab Emirates,January,Abu Dhabi,icon,NaN,2011,2011-01-01
4,acapulco_bts,Acapulco,Mexico,August,Acapulco,bts,Guerrero,2018,2018-08-01


In [32]:
m.sort_values('date', ascending=False).head()

,id,city,country,month,name,series,state,year,date
708,luzhou_yah,Luzhou,China,October,Luzhou,yah,NaN,2019,2019-10-01
785,mianyang_yah,Mianyang,China,October,Mianyang,yah,NaN,2019,2019-10-01
1426,zhuzhou_yah,Zhuzhou,China,October,Zhuzhou,yah,NaN,2019,2019-10-01
1294,trinidad_&_tobago_bts,NaN,Trinidad & Tobago,October,Trinidad & Tobago,bts,NaN,2019,2019-10-01
660,leshan_yah,Leshan,China,October,Leshan,yah,NaN,2019,2019-10-01


In [33]:
# City names adjustments
m['cun'] = m['city']
m.loc[m.cun == 'Washington', 'cun'] = 'Washington DC'
m.loc[m.cun == 'Munster', 'cun'] = 'Muenster'
m.loc[m.cun == 'Aarhus', 'cun'] = 'Arhus'
m.loc[m.cun == 'Acapulco', 'cun'] = 'Acapulco de Juarez'
m.loc[m.cun == 'Antigua', 'cun'] = 'Antigua Guatemala'
m.loc[m.cun == 'Antwerp', 'cun'] = 'Antwerpen'
m.loc[m.cun == 'Bacolod', 'cun'] = 'Bacolod City'
m.loc[m.cun == 'Berne', 'cun'] = 'Bern'
m.loc[m.cun == 'Brunswick', 'cun'] = 'Braunschweig'
m.loc[m.cun == 'Cebu', 'cun'] = 'Cebu City'
m.loc[m.cun == 'Chiangmai', 'cun'] = 'Сhiang Mai'
m.loc[m.cun == 'Cologne', 'cun'] = 'Koeln'
m.loc[m.cun == 'Dusseldorf', 'cun'] = 'Duesseldorf'
m.loc[m.cun == 'Geneva', 'cun'] = 'Geneve'
m.loc[m.cun == 'Gothenburg', 'cun'] = 'Goeteborg'
m.loc[m.cun == 'Hai Phong', 'cun'] = 'Haiphong'
m.loc[m.cun == 'Hong Kong', 'country'] = 'Hong Kong'
m.loc[m.cun == 'Jeju', 'cun'] = 'Jeju City'
m.loc[m.cun == 'Leon', 'cun'] = 'Leon de los Aldama'
m.loc[m.cun == 'Lucerne', 'cun'] = 'Luzern'
m.loc[m.cun == 'Macau', 'country'] = 'Macao'
m.loc[m.cun == 'Makati', 'cun'] = 'Makati City'
m.loc[m.cun == 'Malmo', 'cun'] = 'Malmoe'
m.loc[m.cun == 'Marrakech', 'cun'] = 'Marrakesh'
m.loc[m.cun == 'New York', 'cun'] = 'New York City'
m.loc[m.cun == 'Newcastle Upon Tyne', 'cun'] = 'Newcastle upon Tyne'
m.loc[m.cun == 'Nuremberg', 'cun'] = 'Nuernberg'
m.loc[m.cun == 'Pattaya', 'cun'] = 'Phatthaya'
m.loc[m.cun == 'Playa Del Carmen', 'cun'] = 'Playa del Carmen'
m.loc[m.cun == 'Ras Al Khaimah', 'cun'] = 'Ras al-Khaimah'
m.loc[m.cun == 'Rio De Janeiro', 'cun'] = 'Rio de Janeiro'
m.loc[m.cun == 'Rostov-on-Don', 'cun'] = 'Rostov-na-Donu'
m.loc[m.cun == 'Seville', 'cun'] = 'Sevilla'
m.loc[m.cun == 'St. Gallen', 'cun'] = 'Sankt Gallen'
m.loc[m.cun == 'St. Petersburg', 'cun'] = 'Saint Petersburg'
m.loc[m.cun == 'Taoyuan', 'cun'] = 'Taoyuan City'
m.loc[m.cun == 'Tsingtao', 'cun'] = 'Qingdao'
m.loc[m.cun == 'Xian', 'cun'] = 'Xi\'an'

In [34]:
mc = m.join(c2, on=['cun','country'])

In [35]:
mc.shape

(1431, 19)

In [42]:
mc[['city','country','population']].groupby('city').first().sort_values('population').head()

,country,population
city,,
Corfu,Greece,27003.0
Bodrum,Turkey,39317.0
Antigua,Guatemala,39368.0
Roermond,Netherlands,44975.0
Lucerne,Switzerland,57066.0


In [269]:
#mc[['id','city','country','country code','population']].sort_values('population', ascending=True).head(10)

In [270]:
mg = mc.groupby(['city','country']).first()


In [271]:
mg.loc[mg['population'].isnull()].head(20).tail(10)

,,id,month,name,series,state,year,date,cun,geonameid,latitude,longitude,country code,population,timezone,Capital,Population,Continent
city,country,,,,,,,,,,,,,,,,,
Langkawi,Malaysia,langkawi_yah,November,Langkawi,yah,NaN,2018,2018-11-01,Langkawi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Liuzhou,China,liuzhou_icon,January,Liuzhou,icon,NaN,2013,2013-01-01,Liuzhou,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Los Cabos,Mexico,los_cabos_bts,August,Los Cabos,bts,Baja California,2018,2018-08-01,Los Cabos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Luzhou,China,luzhou_yah,October,Luzhou,yah,NaN,2019,2019-10-01,Luzhou,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ma'anshan,China,maanshan_yah,November,Ma'anshan,yah,NaN,2018,2018-11-01,Ma'anshan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Marne la Vallee,France,disneyland_paris_yah,June,Disneyland Paris,yah,NaN,2018,2018-06-01,Marne la Vallee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mykonos,Greece,mykonos_icon,January,Mykonos,icon,NaN,2011,2011-01-01,Mykonos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Orange County,United States,orange_county_yah,March,Orange County,yah,California,2013,2013-03-01,Orange County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pampanga,Philippines,pampanga_icon,January,Pampanga,icon,NaN,2012,2012-01-01,Pampanga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [272]:
mg.sort_values('population', ascending=True).head()

,,id,month,name,series,state,year,date,cun,geonameid,latitude,longitude,country code,population,timezone,Capital,Population,Continent
city,country,,,,,,,,,,,,,,,,,
Corfu,Greece,corfu_icon,January,Corfu,icon,NaN,2009,2009-01-01,Corfu,2463679.0,39.62069,19.91975,GR,27003.0,Europe/Athens,Athens,11000000.0,EU
Bodrum,Turkey,bodrum_yah,September,Bodrum,yah,NaN,2016,2016-09-01,Bodrum,320995.0,37.03833,27.42917,TR,39317.0,Europe/Istanbul,Ankara,77804122.0,AS
Antigua,Guatemala,antigua_icon,January,Antigua,icon,NaN,2017,2017-01-01,Antigua Guatemala,3599699.0,14.56111,-90.73444,GT,39368.0,America/Guatemala,Guatemala City,13550440.0,NaN
Roermond,Netherlands,roermond_yah,June,Roermond,yah,NaN,2017,2017-06-01,Roermond,2748000.0,51.19417,5.98750,NL,44975.0,Europe/Amsterdam,Amsterdam,16645000.0,EU
Lucerne,Switzerland,lucerne_icon,January,Lucerne,icon,NaN,2009,2009-01-01,Luzern,2659811.0,47.05048,8.30635,CH,57066.0,Europe/Zurich,Bern,8484100.0,EU


In [273]:
c2.loc[c2['population']>30000].shape

(13378, 9)

In [274]:
c3 = c2.loc[c2['population']>30000].drop(columns=['timezone', 'Capital', 'Population', 'Continent']).reset_index()

In [275]:
c3.head()

,city,country,geonameid,latitude,longitude,country code,population
0,'Ain Benian,Algeria,2508275,36.80277,2.92185,DZ,31102
1,'Ain Deheb,Algeria,2508225,34.84218,1.54697,DZ,36146
2,'Ain Merane,Algeria,2508010,36.16277,0.97037,DZ,34413
3,'Ain el Bell,Algeria,2508184,34.34381,3.22475,DZ,31916
4,'Ain el Melh,Algeria,2508130,34.84146,4.16383,DZ,39798


In [276]:
m3 = mg.reset_index()

In [277]:
def hasMug(city: str, country: str):
    return not m3.loc[m3.country==country].loc[m3.city==city].empty

In [278]:
hasMug('New Jersey','United States')

False

In [279]:
c3['hasMug'] = c3[['city','country']].apply(lambda x: hasMug(x['city'],x['country']), axis=1 )

In [280]:
c3.head()

,city,country,geonameid,latitude,longitude,country code,population,hasMug
0,'Ain Benian,Algeria,2508275,36.80277,2.92185,DZ,31102,False
1,'Ain Deheb,Algeria,2508225,34.84218,1.54697,DZ,36146,False
2,'Ain Merane,Algeria,2508010,36.16277,0.97037,DZ,34413,False
3,'Ain el Bell,Algeria,2508184,34.34381,3.22475,DZ,31916,False
4,'Ain el Melh,Algeria,2508130,34.84146,4.16383,DZ,39798,False


In [281]:
c3.loc[c3.hasMug]

,city,country,geonameid,latitude,longitude,country code,population,hasMug
12,Aachen,Germany,3247449,50.77664,6.08342,DE,265208,True
34,Aberdeen,United Kingdom,2657832,57.14369,-2.09814,GB,196670,True
50,Abu Dhabi,United Arab Emirates,292968,24.46667,54.36667,AE,603492,True
86,Adana,Turkey,325363,37.00167,35.32889,TR,1248988,True
352,Alicante,Spain,2521978,38.34517,-0.48149,ES,334757,True
...,...,...,...,...,...,...,...,...
13286,Zhongshan,China,1915223,21.31992,110.57230,CN,3121275,True
13298,Zhuhai,China,1790437,22.27694,113.56778,CN,501199,True
13304,Zhuzhou,China,1783763,27.83333,113.15000,CN,709358,True
13343,Zunyi,China,1783621,27.68667,106.90722,CN,466292,True


In [282]:
#c3.to_csv('c3.csv', index=False)

In [283]:
c3[c3['city'].str.contains('Delhi')]

,city,country,geonameid,latitude,longitude,country code,population,hasMug
3066,Delhi,India,1273294,28.65195,77.23149,IN,10927986,False
8165,New Delhi,India,1261481,28.63576,77.22445,IN,317797,True


In [284]:
m.head()

,id,city,country,month,name,series,state,year,date,cun
0,aachen_icon,Aachen,Germany,January,Aachen,icon,NaN,2009,2009-01-01,Aachen
1,aarhus_icon,Aarhus,Denmark,January,Aarhus,icon,NaN,2014,2014-01-01,Arhus
2,aberdeen_yah,Aberdeen,United Kingdom,August,Aberdeen,yah,Scotland,2017,2017-08-01,Aberdeen
3,abu_dhabi_icon,Abu Dhabi,United Arab Emirates,January,Abu Dhabi,icon,NaN,2011,2011-01-01,Abu Dhabi
4,acapulco_bts,Acapulco,Mexico,August,Acapulco,bts,Guerrero,2018,2018-08-01,Acapulco de Juarez


In [285]:
m[['city','state','country']].head()

,city,state,country
0,Aachen,NaN,Germany
1,Aarhus,NaN,Denmark
2,Aberdeen,Scotland,United Kingdom
3,Abu Dhabi,NaN,United Arab Emirates
4,Acapulco,Guerrero,Mexico


In [286]:
def location_type(city_state_country_name):
    city = city_state_country_name[0]
    state = city_state_country_name[1]
    country = city_state_country_name[2]
    name = city_state_country_name[3]
    
    if 'Disney' in name or 'Universal' in name or 'Magic' in name or 'Epcot' in name or 'Pike' in name:
        return 'special'
    
    location_type = 'country'
    
    if not state != state:
        location_type = 'state'
    
    if not city != city:
        location_type = 'city'
    
    return location_type

In [287]:
m['location_type'] = m[['city','state','country','name']].apply(lambda x: location_type(x), axis = 1)

In [288]:
def location_path(city_state_country):
    city = city_state_country[0]
    state = city_state_country[1]
    country = city_state_country[2]
    
    path = country
    
    if not state != state:
        path = state + ', ' + path
    
    if not city != city:
        path = city +  ', ' + path
    
    return path

In [289]:
m['location_path'] = m[['city','state','country']].apply(lambda x: location_path(x), axis = 1)

In [290]:
m.head()

,id,city,country,month,name,series,state,year,date,cun,location_type,location_path
0,aachen_icon,Aachen,Germany,January,Aachen,icon,NaN,2009,2009-01-01,Aachen,city,"Aachen, Germany"
1,aarhus_icon,Aarhus,Denmark,January,Aarhus,icon,NaN,2014,2014-01-01,Arhus,city,"Aarhus, Denmark"
2,aberdeen_yah,Aberdeen,United Kingdom,August,Aberdeen,yah,Scotland,2017,2017-08-01,Aberdeen,city,"Aberdeen, Scotland, United Kingdom"
3,abu_dhabi_icon,Abu Dhabi,United Arab Emirates,January,Abu Dhabi,icon,NaN,2011,2011-01-01,Abu Dhabi,city,"Abu Dhabi, United Arab Emirates"
4,acapulco_bts,Acapulco,Mexico,August,Acapulco,bts,Guerrero,2018,2018-08-01,Acapulco de Juarez,city,"Acapulco, Guerrero, Mexico"


In [291]:
m[m.name.str.contains('Magic')]

,id,city,country,month,name,series,state,year,date,cun,location_type,location_path
721,magic_kingdom_bto,Lake Buena Vista,United States,July,Magic Kingdom,bts,Florida,2019,2019-07-01,Lake Buena Vista,special,"Lake Buena Vista, Florida, United States"
722,magic_kingdom_bts,Lake Buena Vista,United States,April,Magic Kingdom,bts,Florida,2019,2019-04-01,Lake Buena Vista,special,"Lake Buena Vista, Florida, United States"
723,magic_kingdom_yah,Lake Buena Vista,United States,January,Magic Kingdom,yah,Florida,2015,2015-01-01,Lake Buena Vista,special,"Lake Buena Vista, Florida, United States"
724,magic_kingdom_yah_2,Lake Buena Vista,United States,September,Magic Kingdom v.2,yah,Florida,2017,2017-09-01,Lake Buena Vista,special,"Lake Buena Vista, Florida, United States"
725,magic_kingdom_yah_3,Lake Buena Vista,United States,May,Magic Kingdom v.3,yah,Florida,2018,2018-05-01,Lake Buena Vista,special,"Lake Buena Vista, Florida, United States"
726,magic_kingdom_yaho,Lake Buena Vista,United States,August,Magic Kingdom,yaho,Florida,2016,2016-08-01,Lake Buena Vista,special,"Lake Buena Vista, Florida, United States"
727,magic_kingdom_yaho_2,Lake Buena Vista,United States,November,Magic Kingdom v.2,yaho,Florida,2017,2017-11-01,Lake Buena Vista,special,"Lake Buena Vista, Florida, United States"


In [292]:
m['location_type'].value_counts()

city       914
country    232
state      230
special     55
Name: location_type, dtype: int64

In [303]:
!pip install geopy
from geopy.geocoders import Nominatim

In [327]:
geolocator = Nominatim(user_agent="city_explorer_2")

for index, row in m.iterrows():
    if index > 1215:
        location = geolocator.geocode(row['location_path'])

        if not location is None:
            m.at[index, 'latitude'] = location.latitude
            m.at[index, 'longitude'] = location.longitude


GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

In [325]:
m.head(1220).tail()

,id,city,country,month,name,series,state,year,date,cun,location_type,location_path,latitude,longitude
1215,suzhou_icon,Suzhou,China,January,Suzhou,icon,NaN,2011,2011-01-01,Suzhou,city,"Suzhou, China",31.402994,120.733535
1216,suzhou_rlf,Suzhou,China,January,Suzhou Blue,rlf,NaN,2013,2013-01-01,Suzhou,city,"Suzhou, China",NaN,NaN
1217,suzhou_yah,Suzhou,China,September,Suzhou,yah,NaN,2017,2017-09-01,Suzhou,city,"Suzhou, China",NaN,NaN
1218,sweden_icon,NaN,Sweden,January,Sweden,icon,NaN,2012,2012-01-01,NaN,country,Sweden,NaN,NaN
1219,sweden_yah,NaN,Sweden,September,Sweden,yah,NaN,2018,2018-09-01,NaN,country,Sweden,NaN,NaN


In [326]:
m.to_csv('m_1215.csv', index = False)